# Proyecto 2 entrega #3

## Implementación del modelo de Naive Bayes

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Cargar los datos
data = pd.read_csv('train.csv')

# Convertir la variable objetivo en categorías
num_bins = 3  # Número de categorías
labels = ['bajo', 'medio', 'alto']
data['SalePrice_cat'] = pd.qcut(data['SalePrice'], q=num_bins, labels=labels)

# Seleccionar características y variable objetivo
X = data.drop(['SalePrice', 'SalePrice_cat'], axis=1)
y = data['SalePrice_cat']

# Convertir variables categóricas en numéricas
X = pd.get_dummies(X, drop_first=True)

# Manejo de valores faltantes
X.fillna(X.mean(), inplace=True)  # Rellenar valores numéricos con la media

# Dividir en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenar el modelo Naive Bayes
nb_model = GaussianNB()
nb_model.fit(X_train, y_train)

# Realizar predicciones
y_pred = nb_model.predict(X_test)

# Evaluación del modelo
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

print(f'Precisión del modelo: {accuracy:.4f}')
print('Matriz de confusión:\n', conf_matrix)
print('Reporte de clasificación:\n', class_report)




Precisión del modelo: 0.7637
Matriz de confusión:
 [[ 82   3   9]
 [  0 106   4]
 [ 24  29  35]]
Reporte de clasificación:
               precision    recall  f1-score   support

        alto       0.77      0.87      0.82        94
        bajo       0.77      0.96      0.85       110
       medio       0.73      0.40      0.51        88

    accuracy                           0.76       292
   macro avg       0.76      0.74      0.73       292
weighted avg       0.76      0.76      0.74       292



In [1]:
# MODELO DE CLASIFICACIÓN.
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder

# Cargar los datos
df_train = pd.read_csv("train.csv")

# Crear la variable categórica de precios
q1 = df_train['SalePrice'].quantile(0.25)
q3 = df_train['SalePrice'].quantile(0.75)
df_train['PriceCategory'] = pd.cut(df_train['SalePrice'], bins=[-np.inf, q1, q3, np.inf], labels=['Económicas', 'Intermedias', 'Caras'])

# Seleccionar características para el modelo (ajustar según disponibilidad de datos)
features = ['OverallQual', 'GrLivArea', 'GarageCars', 'TotalBsmtSF', 'LotArea']
X = df_train[features]
y = df_train['PriceCategory']

# Convertir la variable categórica a valores numéricos
le = LabelEncoder()
y = le.fit_transform(y)

# Dividir datos en entrenamiento y prueba
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenar el modelo de clasificación con un árbol de decisión
clf = DecisionTreeClassifier(max_depth=10, random_state=42)
clf.fit(X_train, y_train)

# Predicciones en el conjunto de validación
y_pred = clf.predict(X_val)

# Mostrar el resultado de las predicciones
print("Modelo de clasificación entrenado y listo para evaluación.")


Modelo de clasificación entrenado y listo para evaluación.


In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Cargar datos
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")

q1 = df_train['SalePrice'].quantile(0.25)
q3 = df_train['SalePrice'].quantile(0.75)
df_train['PriceCategory'] = pd.cut(df_train['SalePrice'], bins=[-np.inf, q1, q3, np.inf], labels=['Económicas', 'Intermedias', 'Caras'])

features = ['OverallQual', 'GrLivArea', 'GarageCars', 'TotalBsmtSF', 'FullBath', 'YearBuilt']  # Ajusta si es necesario

le = LabelEncoder()
df_train['PriceCategory'] = le.fit_transform(df_train['PriceCategory'])

X_train = df_train[features]
y_train = df_train['PriceCategory']

clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

missing_cols = [col for col in features if col not in df_test.columns]
if missing_cols:
    raise ValueError(f"Faltan columnas en test.csv: {missing_cols}")

X_test = df_test[features]
y_test_pred = clf.predict(X_test)

if 'SalePrice' in df_test.columns:
    df_test['PriceCategory'] = pd.cut(df_test['SalePrice'], bins=[-np.inf, q1, q3, np.inf], labels=['Económicas', 'Intermedias', 'Caras'])
    df_test['PriceCategory'] = le.transform(df_test['PriceCategory'])  # Convertir a valores numéricos

    accuracy = accuracy_score(df_test['PriceCategory'], y_test_pred)
    report = classification_report(df_test['PriceCategory'], y_test_pred, target_names=le.classes_)

    print(f"\n🔹 Exactitud del modelo: {accuracy:.4f}")
    print("\n🔹 Reporte de clasificación:\n", report)

df_test['PredictedCategory'] = le.inverse_transform(y_test_pred)
df_test.to_csv("test_predicted.csv", index=False)

print("✅ Predicciones generadas y guardadas en 'test_predicted.csv'.")


✅ Predicciones generadas y guardadas en 'test_predicted.csv'.


In [3]:
from sklearn.model_selection import cross_val_score

accuracy = cross_val_score(clf, X_train, y_train, cv=5, scoring='accuracy')
print(f"🔹 Exactitud promedio con validación cruzada: {accuracy.mean():.4f}")

🔹 Exactitud promedio con validación cruzada: 0.8123


In [4]:
import pandas as pd
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

# Cargar el archivo 'train.csv' y 'test_predicted.csv'
train_df = pd.read_csv("train.csv")
test_pred_df = pd.read_csv("test_predicted.csv")

# Asegurarse de que las columnas necesarias estén en los datos
if 'SalePrice' not in train_df.columns:
    raise ValueError("El archivo 'train.csv' no contiene la columna 'SalePrice'.")
if 'PredictedCategory' not in test_pred_df.columns:
    raise ValueError("El archivo 'test_predicted.csv' no contiene la columna 'PredictedCategory'.")

# Para efectos de este ejemplo, vamos a suponer que la columna 'PredictedCategory' es una categoría de precio (e.g., Económicas, etc.)
# Si en realidad es un valor numérico, tendríamos que ajustarlo según las categorías
le = LabelEncoder()

# Si la columna 'PredictedCategory' contiene categorías no numéricas, codificarlas
test_pred_df['PredictedCategory'] = le.fit_transform(test_pred_df['PredictedCategory'])

# Para el análisis, vamos a usar las etiquetas reales de 'SalePrice' y generar categorías
# Convertimos 'SalePrice' en categorías (por ejemplo, alto, medio, bajo) según un umbral
price_threshold = train_df['SalePrice'].median()  # Usamos la mediana como umbral

# Crear una nueva columna categórica en 'train_df' para clasificar el precio
train_df['PriceCategory'] = train_df['SalePrice'].apply(lambda x: 'High' if x > price_threshold else 'Low')

# Codificar 'PriceCategory' en el mismo formato que 'PredictedCategory'
train_df['PriceCategory'] = le.fit_transform(train_df['PriceCategory'])

# Alinear las predicciones con las etiquetas reales
y_true = train_df['PriceCategory']  # Etiquetas reales
y_pred = test_pred_df['PredictedCategory']  # Predicciones

# Calcular la matriz de confusión
cm = confusion_matrix(y_true, y_pred)

# Mostrar la matriz de confusión usando un mapa de calor
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=le.classes_, yticklabels=le.classes_)
plt.title("Matriz de Confusión")
plt.xlabel("Predicción")
plt.ylabel("Valor Real")
plt.show()

# Calcular la exactitud (accuracy) y el reporte de clasificación
accuracy = accuracy_score(y_true, y_pred)
report = classification_report(y_true, y_pred, target_names=le.classes_)

print(f"\n🔹 Exactitud del modelo: {accuracy:.4f}")
print("\n🔹 Reporte de clasificación:\n", report)

# Análisis de los errores:
# Total de errores por clase:
class_errors = cm.sum(axis=1) - np.diagonal(cm)  # Total de errores por clase

print("\n🔹 Errores por clase:")
for i, class_error in enumerate(class_errors):
    print(f"Clase '{le.classes_[i]}': {class_error} errores")

# Evaluación de la efectividad: Identificar dónde el modelo se equivocó más y menos
most_missed_class = np.argmax(class_errors)
least_missed_class = np.argmin(class_errors)

print(f"\n🔹 Donde más se equivocó el modelo: {le.classes_[most_missed_class]} (errores: {class_errors[most_missed_class]})")
print(f"🔹 Donde menos se equivocó el modelo: {le.classes_[least_missed_class]} (errores: {class_errors[least_missed_class]})")

# Calcular la importancia de los errores: se puede evaluar el número de errores por clase en función del total de instancias
total_instances = cm.sum()
error_percentage_by_class = (class_errors / total_instances) * 100

print("\n🔹 Importancia de los errores por clase (en porcentaje):")
for i, error_percent in enumerate(error_percentage_by_class):
    print(f"Clase '{le.classes_[i]}': {error_percent:.2f}% de los errores")



ValueError: Found input variables with inconsistent numbers of samples: [1460, 1459]

## Verificar si está sobreajustado y modelo de validación cruzada

In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Cargar los datos
data = pd.read_csv('train.csv')

# Convertir la variable objetivo en categorías
num_bins = 3  # Número de categorías
labels = ['bajo', 'medio', 'alto']
data['SalePrice_cat'] = pd.qcut(data['SalePrice'], q=num_bins, labels=labels)

# Seleccionar características y variable objetivo
X = data.drop(['SalePrice', 'SalePrice_cat'], axis=1)
y = data['SalePrice_cat']

# Convertir variables categóricas en numéricas
X = pd.get_dummies(X, drop_first=True)

# Manejo de valores faltantes
X.fillna(X.mean(), inplace=True)  # Rellenar valores numéricos con la media

# Dividir en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenar el modelo Naive Bayes
nb_model = GaussianNB()
nb_model.fit(X_train, y_train)

# Precisión en el conjunto de entrenamiento
y_train_pred = nb_model.predict(X_train)
train_accuracy = accuracy_score(y_train, y_train_pred)

# Realizar predicciones en el conjunto de prueba
y_pred = nb_model.predict(X_test)
test_accuracy = accuracy_score(y_test, y_pred)

# Validación cruzada
cv_scores = cross_val_score(nb_model, X, y, cv=5)
cv_mean = np.mean(cv_scores)

# Evaluación del modelo
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

print(f'Precisión en entrenamiento: {train_accuracy:.4f}')
print(f'Precisión en prueba: {test_accuracy:.4f}')
print(f'Precisión con validación cruzada (media de 5 folds): {cv_mean:.4f}')
print('Matriz de confusión:\n', conf_matrix)
print('Reporte de clasificación:\n', class_report)


Precisión en entrenamiento: 0.7466
Precisión en prueba: 0.7637
Precisión con validación cruzada (media de 5 folds): 0.7397
Matriz de confusión:
 [[ 82   3   9]
 [  0 106   4]
 [ 24  29  35]]
Reporte de clasificación:
               precision    recall  f1-score   support

        alto       0.77      0.87      0.82        94
        bajo       0.77      0.96      0.85       110
       medio       0.73      0.40      0.51        88

    accuracy                           0.76       292
   macro avg       0.76      0.74      0.73       292
weighted avg       0.76      0.76      0.74       292



## Informe

### Análisis del modelo
* **Precisión Global**: 76.37% de precisión significa que el modelo claasificó correctamente alrededor del 76% de los valores conjuntos de prueba. Para un modelo de Naive Bayes, es un resultado aceptable.
* **Matriz de confusión**: 
    * Clase **alto**: 82 predicciones correctas, 12 incorrectas
    * Clase **media**: Buena precisión y recall alto, lo que significa que casi todos los casos reales de "bajo" fueron clasificados correctamente
    * Clase **medio**: Tiene la peor rendimiento con recall de 40% indicando que el modelo tiene problemas para identificar la categoría.
### Comparación de los modelos
* **Modelo Regresión Lineal**: Tiene un 79.9% de precisión, tiene los coeficientes más estables.
* **Modelo Árbol de Regresión**: Precisión de 79.58%, este siendo de produndidad 10.
* **Modelo Naive Bayes**: Precisión de 76.37%, valor aceptable

En general el modelo de regresión lineal ajustado que se hizo con anterioridad sigue siendo el mejor modelo para la predicción.

### Análisis Sobreajustamiento y Modelo de validación cruzada
* La precisión del entrenamiedo fue de 74.66% y la precisión en prueba de 76.37%, como no es una diferencia signidicativa entonces el modelo **no** esta sobreajustado.
* La precisión de la validación cruzada fue de 73.97%, es ligeramente menor que la precisión en prueba pero en general indica que el modelo es estable y generaliza bien a distintos conjuntos de datos.

### Conclusiones
* El modelo Naive Bayes predice con buena presición las categorias "alto" y "bajo"
* El modelo no está sobreajustado
* No hay una gran diferencia entre entrenamiento, prueba y validación cruzada.